In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [89]:
# top_mov = ['', '?offset=200', '?offset=400', '?offset=600', '?offset=800', '?offset=1000', '?offset=1200', '?offset=1400', '?offset=1600', '?offset=1800']
# list_of_titles = []
# list_of_revenue = []
# for i in top_mov:
#     rev_url = 'https://www.boxofficemojo.com/chart/top_lifetime_gross/{}'.format(i)
#     rev_page = requests.get(rev_url)
#     rsoup = BeautifulSoup(rev_page.content, 'html.parser')
#     rev_title = rsoup.find_all('td', class_='a-text-left mojo-field-type-title')
# #                             ('a', class_='a-link-normal')
#     practice_titles =[]
#     for title in rev_title:
#         practice_titles.append(title.text)    
#     list_of_titles_and_years = practice_titles#[15:-11]
# #     list_of_titles_and_years
#     for value in list_of_titles_and_years:
#         #if value < "1000" or value > "2020": 
#         list_of_titles.append(value)
#     rev_money = rsoup.find_all('td', class_='a-text-right mojo-field-type-money')
# #     practice_money = []
#     for revenue in rev_money:
#         list_of_revenue.append(revenue.text)

In [78]:
len(list_of_revenue)

In [77]:
len(list_of_titles)

In [74]:
top_gross = dict(zip(list_of_titles,list_of_revenue))
# top_gross

In [15]:
# creates 2 lists of top 2000 movie titles and their opening weekend for G/PG rated movies
rated_pg = ['', '&offset=200', '&offset=400', '&offset=600', '&offset=800', '&offset=1000', '&offset=1200', '&offset=1400', '&offset=1600', '&offset=1800']
# list_of_titles = []
# list_of_revenue = []
list_of_pgmovies = []
list_of_pgrevenue = []
for i in rated_pg:
    pg_url = 'https://www.boxofficemojo.com/chart/top_opening_wknd_by_mpaa/?by_mpaa=G%2FPG{}'.format(i)
    pg_page = requests.get(pg_url)
    rsoup = BeautifulSoup(pg_page.content, 'html.parser')
    pg_title = rsoup.find_all('td', class_='a-text-left mojo-header-column mojo-truncate mojo-field-type-release mojo-cell-wide')
#                             ('a', class_='a-link-normal')
    practice_pg =[]
    for title in pg_title:
        practice_pg.append(title.text)    
    working_list_of_pgmovies = practice_pg#[15:-11]
#     list_of_titles_and_years
    for value in working_list_of_pgmovies:
        #if value < "1000" or value > "2020": 
        list_of_pgmovies.append(value)
    pg_money = rsoup.find_all('td', class_='a-text-right mojo-field-type-money mojo-estimatable')
#     practice_money = []
    for revenue in pg_money:
        list_of_pgrevenue.append(revenue.text)

In [14]:
len(list_of_pgmovies)

2000

In [18]:
len(list_of_pgrevenue[::3])

2000